In [1]:
import polars as pl
import os
import numpy as np
import torch, torchvision
from torchvision.utils import make_grid
from glio.loaders import nifti
from glio.python_tools import get_all_files
from glio.jupyter_tools import clean_mem
import matplotlib.pyplot as plt

In [2]:
path = r"E:\dataset\PKG - UPENN-GBM-NIfTI\UPENN-GBM"
os.listdir(path)

['NIfTI-files',
 'radiology_mapping.csv',
 'radiomic_features_CaPTk',
 'UPENN-GBM_acquisition.csv',
 'UPENN-GBM_CaPTk_fe_params.csv',
 'UPENN-GBM_CaPTk_radiomic_features_list.csv',
 'UPENN-GBM_clinical_info_v2.1.csv',
 'UPENN-GBM_data_availability.csv']

In [3]:
df = pl.read_csv(f"{path}/UPENN-GBM_clinical_info_v2.1.csv", null_values = ["Not Available", "NA"]).drop_nulls("PsP_TP_score")
df

ID,Gender,Age_at_scan_years,Survival_from_surgery_days_UPDATED,Survival_Status,Survival_Censor,IDH1,MGMT,KPS,GTR_over90percent,Time_since_baseline_preop,PsP_TP_score
str,str,f64,i64,str,i64,str,str,i64,str,i64,i64
"""UPENN-GBM-0003…","""M""",43.57,286,"""Deceased""",null,"""Wildtype""",null,null,"""Not Applicable…",521,6
"""UPENN-GBM-0004…","""M""",62.83,76,"""Deceased""",null,"""NOS/NEC""",null,null,"""Not Applicable…",282,6
"""UPENN-GBM-0004…","""F""",51.11,419,"""Deceased""",null,"""NOS/NEC""",null,null,"""Not Applicable…",113,2
"""UPENN-GBM-0005…","""F""",61.26,401,"""Deceased""",null,"""Wildtype""",null,null,"""Not Applicable…",385,6
"""UPENN-GBM-0005…","""M""",54.34,183,"""Deceased""",null,"""Wildtype""",null,null,"""Not Applicable…",362,6
…,…,…,…,…,…,…,…,…,…,…,…
"""UPENN-GBM-0062…","""F""",57.19,288,"""Deceased""",null,"""Wildtype""","""Unmethylated""",null,"""Not Applicable…",234,5
"""UPENN-GBM-0062…","""F""",57.4,365,"""Deceased""",null,"""Wildtype""",null,null,"""Not Applicable…",null,4
"""UPENN-GBM-0062…","""M""",55.09,79,"""Deceased""",null,"""Wildtype""",null,null,"""Not Applicable…",null,6


In [19]:
def loader(idx:int):
    row = df.row(idx)
    study_id = row[0]
    PsP_TP_score = row[-1]
    path = rf"E:\dataset\PKG - UPENN-GBM-NIfTI\UPENN-GBM\NIfTI-files\images_DTI\images_DTI\{study_id}\{study_id}_DTI_AD.nii.gz"
    return nifti.read(path).unsqueeze(0)

In [7]:
files = get_all_files(r"E:\dataset\PKG - UPENN-GBM-NIfTI\UPENN-GBM\NIfTI-files\images_DTI\images_DTI", path_filter=lambda x: "DTI_AD" in x)
for f in files:
    img = nifti.read(f)[44:191, 36:222, 2:141]
    print(img.min(), img.max(), img.mean(), img.std())

tensor(0.) tensor(0.0050) tensor(0.0005) tensor(0.0007)
tensor(0.) tensor(0.0036) tensor(0.0004) tensor(0.0006)
tensor(0.) tensor(0.0044) tensor(0.0005) tensor(0.0007)
tensor(0.) tensor(0.0046) tensor(0.0005) tensor(0.0007)
tensor(0.) tensor(0.0048) tensor(0.0005) tensor(0.0007)
tensor(0.) tensor(0.0045) tensor(0.0005) tensor(0.0007)
tensor(0.) tensor(0.0035) tensor(0.0004) tensor(0.0006)
tensor(0.) tensor(0.0036) tensor(0.0003) tensor(0.0006)
tensor(0.) tensor(0.0042) tensor(0.0006) tensor(0.0008)
tensor(0.) tensor(0.0050) tensor(0.0007) tensor(0.0009)
tensor(0.) tensor(0.0048) tensor(0.0005) tensor(0.0007)
tensor(0.) tensor(0.0043) tensor(0.0005) tensor(0.0007)
tensor(0.) tensor(0.0052) tensor(0.0005) tensor(0.0008)
tensor(0.) tensor(0.0043) tensor(0.0005) tensor(0.0007)
tensor(0.) tensor(0.0045) tensor(0.0006) tensor(0.0008)
tensor(0.) tensor(0.0047) tensor(0.0005) tensor(0.0007)
tensor(0.) tensor(0.0034) tensor(0.0005) tensor(0.0007)
tensor(0.) tensor(0.0040) tensor(0.0004) tensor(

In [21]:
from glio.data.DS import DSBasic
ds = DSBasic()
ds.add_samples(list(range(len(df))), loader=loader)
mean_std = ds.get_mean_std(1000, 16, 0)
mean_std

(tensor([0.0002]), tensor([0.0005]))

In [22]:
torch.set_printoptions(precision=10)
mean_std

(tensor([0.0002149488]), tensor([0.0005426282]))

In [23]:
ds.add_loader(torchvision.transforms.Normalize(*mean_std))

In [25]:
ds.samples[0].loader.transforms

(<function __main__.loader(idx: int)>,
 Normalize(mean=tensor([0.0002149488]), std=tensor([0.0005426282])))

In [26]:
mean_std = ds.get_mean_std(1000, 16, 0)
mean_std

(tensor([0.0005027046]), tensor([1.0011870861]))

In [13]:
import polars as pl
path = r"E:\dataset\PKG - UPENN-GBM-NIfTI\UPENN-GBM"
df = pl.read_csv(f"{path}/UPENN-GBM_clinical_info_v2.1.csv", null_values = ["Not Available", "NA"]).drop_nulls("PsP_TP_score")
for row in df.rows():
    print(row[-1])

6
6
2
6
6
5
2
4
5
5
4
6
3
5
5
5
5
4
3
1
5
6
6
6
6
2
2
6
5
6
1
4
1
3
3
4
5
4
3
4
3
6
6
4
6
6
6
6
3
5
5
6
3
5
3
5
4
6
6
2
